#### Day 6 - B
Navigate the grid based on the rules:
* If there is an obstacle directly ahead, turn to the right.
* If there is open space directly ahead, move forward.

Add an obstruction to create a loop.
Count number of possible obstructions.

In [ ]:
#Import Libraries and settings
from copy import deepcopy

settings = {
    "day": 6,
    "test_data": 0
}

In [ ]:
#Load Input
def load_input(settings):
    #Derrive input file name
    if settings["test_data"]:
        data_subdir = "test"
    else:
        data_subdir = "actual"

    data_fp = f"./../input/{data_subdir}/{settings["day"]}.txt"

    #Open and read the file
    with open(data_fp) as f:
        lines = f.read().split('\n')

    grid = [list(x) for x in lines]

    return grid

In [ ]:
#Get the starting location for a grid (look for arrow character)
def get_starting_loc(grid, dir_start="^", obs=None):
    #Iterate through grid to find the starting character
    for idx_y, line in enumerate(grid):
        for idx_x, space in enumerate(line):
            if space == dir_start:
                return (idx_x, idx_y), dir_start

    #Error
    print(obs)
    for line in grid:
        print(line)
    raise Exception("No location found")

#Convert compass direction into a dir tuple
def translate_step(dir_g, mag=1):
    v = 0
    h = 0

    if "^" in dir_g:
        v = -mag
    elif "v" in dir_g:
        v = mag

    if ">" in dir_g:
        h = mag
    elif "<" in dir_g:
        h = -mag

    return (h, v)

#Return location after applying dir
def apply_dir(loc, idx_dir):

    return (loc[0] + idx_dir[0], loc[1] + idx_dir[1])

#From a given location, take a step and update the grid
def take_step (grid, loc, new_loc):

    #Replace current loc with X
    #grid[loc[1]][loc[0]] = "X"

    #Update new location
    grid[new_loc[1]][new_loc[0]] = "X"

#Get the new location
def get_new_loc(loc, dir_g):
    new_loc = apply_dir(loc, translate_step(dir_g))
    return new_loc

#Check a new location in the grid
def check_new_loc(grid, loc, dir_g, visited):

    #Check for loop
    state = (loc, dir_g)
    if state in visited:
        return "l"

    try:
        #Check if loc index is negative (won't throw error when accessing)
        if loc[0] < 0 or loc[1] < 0:
            return "o"

        new_char = grid[loc[1]][loc[0]]
        return new_char

    except:
        #Location does not exist inside the grid
        return "o"

def apply_turn(dir_g):
    dirs = ["^", ">", "v", "<", "^"]
    current_dir = dirs.index(dir_g)
    new_dir = dirs[current_dir + 1]
    return new_dir

#Count the number of visited spaces ("X") in the grid
def count_visited_spaces(grid):
    count = 0
    for line in grid:
        for space in line:
            if space == "X":
                count += 1

    return count

#Add an obstacle char in the given location
def add_obs(grid, loc):
    grid[loc[1]][loc[0]] = "#"

#Function to handle the grid steps, end condition and result
def grid_controller(grid, obs=False, obs_loc=None, loc_s=False, dir_s = None):

    if obs:
        #Count obs locations
        obs_count = 0
        obs_tested = []
    else:
        #Add the obs to the specified location
        add_obs(grid, obs_loc)
    visited = []

    #Get initial starting direction
    if loc_s == False:
        loc_s, dir_s = get_starting_loc(grid, obs=obs)

    loc = loc_s
    dir_g = dir_s

    #Update starting location as a visited space
    #grid[loc[1]][loc[0]] = "X"

    while True:
        new_loc = get_new_loc(loc, dir_g)
        new_char = check_new_loc(grid, new_loc, dir_g, visited)

        #Stepping ends when leaving the grid
        if new_char == "o":
            break
        #Check if new location is an obstacle
        elif new_char == "#":
            dir_g = apply_turn(dir_g)
        #Check if stuck in a loop
        elif new_char == "l":
            return True

        #Otherwise take the step
        else:
            take_step(grid, loc, new_loc)
            loc = new_loc

            #Update visited states
            if not obs:
                visited.append((loc, dir_g))
                

            if obs and loc not in obs_tested:
                #Test obstruction in this location
                res = grid_controller(deepcopy(grid), obs=False, obs_loc=loc, loc_s=loc_s, dir_s=dir_s)
                if res:
                    obs_count += 1
                    obs_tested.append(loc)

                    if obs_count % 100 == 0:
                        print("Status: Loc-", loc, " Obs Count-", obs_count)

    #Left the grid so not in an obstruction
    if obs == False:
        return False
    else:
        return obs_count

In [ ]:
#Solution takes 57 Minutes to finish
data_in = load_input(settings)
res = grid_controller(data_in, obs=True)

print(res)
# for line in new_grid:
#     print(''.join(line))